In [12]:
from objects import BallOnPlate

import random

from deap import base
from deap import creator
from deap import tools
from deap import algorithms

import numpy
import multiprocessing

import os
import sys

import pickle
import time

import neat

from time import sleep
import math
import numpy as np

simulation_seconds = 10.

def eval_genome(genome):
    ballOnPlate = BallOnPlate(showGUI=False, randomInitial=False)

    global_cost = 1e10

    CONST_VALUE = 0.7
    intial_positions = [[CONST_VALUE, CONST_VALUE],
                        [-CONST_VALUE, -CONST_VALUE],
                        [-CONST_VALUE, CONST_VALUE],
                        [CONST_VALUE, -CONST_VALUE],
                        [0., 0.]]

    reference_positions = [[-CONST_VALUE, -CONST_VALUE],
                           [CONST_VALUE, CONST_VALUE],
                           [CONST_VALUE, -CONST_VALUE],
                           [-CONST_VALUE, CONST_VALUE],
                           [0., 0.]]

    for i in range(len(intial_positions)):

        envInput = [0, 0]
        result = 0
        dropDown = False

        ref_point  = np.array(reference_positions[i])

        posOnPlate = ballOnPlate.reset(np.array(intial_positions[i]))
        prevPosOnPlate = posOnPlate

        prev_err    = [0, 0]
        integr_err  = 0

        ### PID controller
        prop    = float(genome[0])
        diff    = float(genome[1]) / BallOnPlate.D_T
        integr  = float(genome[2]) * BallOnPlate.D_T

        while ballOnPlate.time < simulation_seconds:
            # Get error
            err = ref_point - posOnPlate
            result -= (err[0] * err[0] + err[1] * err[1]) * (ballOnPlate.time + 1)

            integr_err += err
            d_err = err - prev_err

            envInput[0] = prop * err[1] + diff * d_err[1] + integr_err[1] * integr
            envInput[0] = -envInput[0]
            envInput[1] = prop * err[0] + diff * d_err[0] + integr_err[0] * integr

            prev_err = err
            ### PID controller

            prevPosOnPlate = posOnPlate

            posOnPlate, isEnd = ballOnPlate.step(envInput)
            if isEnd:
                # Bad penalty as fall
                dropDown = True
                break
            # sleep(ballOnPlate.dt)
        
        DROP_PENALTY_VALUE = 1e4 

        if dropDown:
            current_cost = (ballOnPlate.time * (DROP_PENALTY_VALUE/simulation_seconds) + result) - DROP_PENALTY_VALUE
        else:
            current_cost = result

#         global_cost += current_cost
        global_cost = min(current_cost, global_cost)

    ballOnPlate.close()
#     global_cost /= float(len(intial_positions))
    return global_cost,

def checkBoundsZero():
    def decorator(func):
        def wrapper(*args, **kargs):
            offspring = func(*args, **kargs)
            for child in offspring:
                for i in xrange(len(child)):
                    if child[i] < 0:
                        child[i] = 0
            return offspring
        return wrapper
    return decorator


In [13]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# Attribute generator 
#                      define 'attr_bool' to be an attribute ('gene')
#                      which corresponds to integers sampled uniformly
#                      from the range [0,1] (i.e. 0 or 1 with equal
#                      probability)
# toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("attr_float", random.random)

# Structure initializers
#                         define 'individual' to be an individual
#                         consisting of 100 'attr_bool' elements ('genes')
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_float, 3)

# define the population to be a list of individuals
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


#----------
# Operator registration
#----------
# register the goal / fitness function
toolbox.register("evaluate", eval_genome)

# register the crossover operator
toolbox.register("mate", tools.cxUniform, indpb=0.7)

# register a mutation operator with a probability to
# flip each attribute/gene of 0.05
toolbox.register("mutate", tools.mutGaussian, indpb=0.1, mu=0, sigma=.02)

toolbox.decorate("mate", checkBoundsZero())
toolbox.decorate("mutate", checkBoundsZero())

# operator for selecting individuals for breeding the next
# generation: each individual of the current generation
# is replaced by the 'fittest' (best) of three individuals
# drawn randomly from the current generation.
toolbox.register("select", tools.selTournament, tournsize=7)

In [ ]:
random.seed(time.time())

# Process Pool of 4 workers
pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
toolbox.register("map", pool.map)

pop = toolbox.population(n=50)
hof = tools.HallOfFame(5)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", numpy.mean)
stats.register("std", numpy.std)
stats.register("min", numpy.min)
stats.register("max", numpy.max)

try:
    algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.3, ngen=10000, 
                        stats=stats, halloffame=hof)
except KeyboardInterrupt:
    print('Try-catch')

pool.close()

print("-- End of (successful) evolution --")

best_ind = tools.selBest(pop, 1)[0]
print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))


gen	nevals	avg     	std   	min     	max     
0  	50    	-9144.11	7.3676	-9147.65	-9113.06
1  	27    	-9129.04	20.5131	-9147.5 	-9010.7 
2  	29    	-9089.75	47.3259	-9147.45	-9010.1 
3  	34    	-9028.79	39.5033	-9113.06	-9001.33
4  	32    	-9009.14	2.89108	-9010.7 	-9001.33
5  	38    	-9003.16	17.0794	-9019.06	-8888.31
6  	37    	-8987.08	36.7988	-9001.37	-8888.24
7  	33    	-8941.34	57.1259	-9078.89	-8888.24
8  	34    	-8887.52	3.00163	-8888.33	-8870.68
9  	22    	-8883.21	12.1568	-8888.29	-8807.54
10 	32    	-8873.05	28.3764	-9010.86	-8790.64
11 	30    	-8830.48	36.851 	-8915.27	-8790.64
12 	31    	-8791.97	63.2434	-9036.26	-8422.49
13 	38    	-8778.33	78.0354	-9010.55	-8422.49
14 	38    	-8723.56	132.438	-8897.35	-8422.49
15 	28    	-8537   	176.901	-9087.09	-8382.68
16 	33    	-8417.98	12.2395	-8422.49	-8382.68
17 	36    	-8416.53	110.769	-9180.67	-8382.25
18 	31    	-8412.54	145.571	-9175.31	-8382.25
19 	36    	-8400.02	97.8407	-9053.94	-8382.25
20 	29    	-8381.71	6.13241	-8406.08